In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
train_X, train_Y, test_X, test_Y = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [18]:
learning_rate = 0.001
training_steps = 1000
batch_size = 128
display_step = 100

In [6]:
num_input = 28 
timesteps = 28 
num_hidden = 128 
num_classes = 10 

In [7]:
weights = {
    'out': tf.Variable(tf.random_normal([2*num_hidden,num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
def BiRNN(X, weights, biases):
    X = tf.unstack(X,timesteps,1)
    
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden,forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden,forget_bias=1.0)
    
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, X,
                                              dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [9]:
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [10]:
logits = BiRNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

In [20]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_op)

In [21]:
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 100, Minibatch Loss= 2.1898, Training Accuracy= 0.234
Step 200, Minibatch Loss= 2.1853, Training Accuracy= 0.258
Step 300, Minibatch Loss= 1.9960, Training Accuracy= 0.320
Step 400, Minibatch Loss= 2.0192, Training Accuracy= 0.250
Step 500, Minibatch Loss= 1.8655, Training Accuracy= 0.430
Step 600, Minibatch Loss= 1.8020, Training Accuracy= 0.391
Step 700, Minibatch Loss= 1.7894, Training Accuracy= 0.430
Step 800, Minibatch Loss= 1.8477, Training Accuracy= 0.312
Step 900, Minibatch Loss= 1.7106, Training Accuracy= 0.445
Step 1000, Minibatch Loss= 1.5652, Training Accuracy= 0.477
Optimization Finished!
Testing Accuracy: 0.4609375
